In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import openflexure_microscope_client as ofm_client
from PIL import Image
import imutils
import seabreeze
seabreeze.use("cseabreeze")
from seabreeze.spectrometers import list_devices, Spectrometer
plt.ioff()
import os
import time

In [ ]:
#check that the spectrometer has been connected
devices =  list_devices()
devices

In [ ]:
# Connect to the microscope
microscope = ofm_client.MicroscopeClient("169.254.193.20")

In [ ]:
#This function takes an image taken by the microscope and segments it into foreground and background

def segmentation_function(image_path, foreground_name):
	# Arguments
	# image_path: the image path of the file that you want to segment
	# foreground_name: the name of the file that the segmented foreground image will be saved to
	
	# Load in an image
	img = cv.imread(image_path)
	assert img is not None, "file could not be read, check with os.path.exists()"
	# Threshold the image
	gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
	ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

	# noise removal
	kernel = np.ones((3,3),np.uint8)
	opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
	# sure background area
	sure_bg = cv.dilate(opening,kernel,iterations=3)
	# Finding sure foreground area
	dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
	ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
	# Finding unknown region
	sure_fg = np.uint8(sure_fg)
	unknown = cv.subtract(sure_bg,sure_fg)
# I've found that the best results come when adding the sure foreground and unknown region(not sure why)
	actual_fg = cv.add(sure_fg, unknown)

	# Save the segmented image to check that the foreground and background have been separarted as desired
    # If it hasn't been successful I find that changing the size of the noise removal matrix can yield better results depending on the density of features in the image
    
		  
	cv.imwrite(foreground_name, actual_fg)

	return 

In [ ]:
# these functions take and save spectra
import json

# the integration time has to be set so that the spectrometer doesn't saturate
# 19000 was the maximum non-staurating value for the spectrometer I used
spec = Spectrometer(devices[0])
spec.integration_time_micros(19000)

def take_spectrum(spec, filename, plotname):
    """Take and plot a spectrum"""
    spectrum = spec.spectrum()
    f, ax = plt.subplots(1,1)
    ax.plot(*spectrum)
    ax.set_xlabel("Wavelength/nm")
    ax.set_ylabel("Counts")
    np.savetxt(filename, spectrum.T, header="Wavelength/nm, Counts", delimiter=", ")
    plt.savefig(plotname)
    return spectrum

def save_spectrum(spectrum, filename):
    """save a spectrum to a csv file"""
    np.savetxt(filename, spectrum.T, header="Wavelength/nm, Counts", delimiter=", ")

# disabling the thermo-electrice cooler of the spectrometer can resolve power issues
try:
    tec = spec.features["thermo_electric"][0]
    tec.enable_tec(False)
    print("Disabled thermo-electric cooler.")
except KeyError as e:
    print(f"Couldn't disable TEC, probably it's not present ({e})")
wl = spec.wavelengths()
print(f"Spectrometer range is {np.min(wl)}--{np.max(wl)}nm")

In [ ]:
# Baseline spectra are required so that spectra of features can be analysed
# Make sure there is no sample on the stage before running this code
take_spectrum(spec, 'white_spec', 'white_spec_plot.png')

In [ ]:
# Now disable the illumination (manually) to take a dark spectrum
take_spectrum(spec, 'dark_spec', 'dark_spec_plot.png')

In [ ]:
# This code loads the background spectra for use later
with open('white_spec') as f:
    lines = (line for line in f if not line.startswith('#'))
    specwhitedata = np.loadtxt(lines, delimiter=',', skiprows=1)

with open('dark_spec') as f:
    lines = (line for line in f if not line.startswith('#'))
    specdarkdata = np.loadtxt(lines, delimiter=',', skiprows=1)

In [ ]:
# now we can define a function that processes the spectra of features
def process_spec(measured_spec):
    with open(measured_spec) as f:
        lines = (line for line in f if not line.startswith('#'))
        spec_data = np.loadtxt(lines, delimiter=',', skiprows=1)
    #now the data can be accessed to make a processed spectrum plot and data file
    #second column of the arrays contains the number of counts at the spectrometer
    specblob = spec_data[:,1]
    specwhite = specwhitedata[:,1]
    specdark = specdarkdata[:,1]
    #the wavelength data is held in the first column
    wavelengths = spec_data[:,0]
    #now the spectrum can be processed
    processed_spec = (specblob-specdark)/(specwhite-specdark)
    #This plot is of the processed spectrum
    plt.plot(wavelengths,processed_spec)
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Relative Intensity')
    plt.axis([300,1000,-1,5])
    plt.savefig('processed_spectrum_{0}.png'.format(measured_spec))
    #also save the processed spectrum as a .csv file
    spectrum_with_counts = np.vstack((wavelengths, processed_spec))
    np.savetxt('{0}_processed.csv'.format(measured_spec), spectrum_with_counts.T, header='Wavelength (nm), Counts', delimiter = ',')

In [ ]:
# focus the microsocpe and take an image
microscope.autofocus()
time.sleep(0.5)
image = microscope.grab_image_array()

#it's useful to view the un-segmented image
plt.imshow(image)

In [ ]:
# now segment the image and view how well the segementation algorithm has worked
fname = 'cv_image.png'
foreground_name = 'segmented.png'
cv.imwrite(fname, image)
segmentation_function(fname, foreground_name)
fg_3channel = cv.imread(foreground_name)
fg = cv.cvtColor(fg_3channel, cv.COLOR_BGR2GRAY)
plt.imshow(fg)

In [ ]:
# draw contours around each feature to find the central point of the feature

cnts = cv.findContours(fg.copy(), cv.RETR_EXTERNAL,
    cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
annotated = fg.copy()

template_size = 100
centres = []
for c in cnts:
    # compute the centre of the contour
    M = cv.moments(c)
    if M['m00'] ==0:
        continue
    cX = int(M["m10"] / M["m00"])		
    cY = int(M["m01"] / M["m00"])
# find the central point of each contour
    cv.drawContours(annotated, [c], -1, (0, 255, 0), 2)
    cv.circle(annotated, (cX, cY), 7, (0,0,0), -1)
    if cX < template_size//2 or cY < template_size//2:
        continue
    if fg.shape[1] - cX < template_size//2 or fg.shape[0] - cY < template_size//2:
        continue
    centres.append((cX, cY))

# now we plot the computed central points on the original image
# this is a useful visual check that the algorithm has worked and looks good for presentation purposes

plt.imshow(image)
plt.plot(np.array(centres)[:,0], np.array(centres)[:,1], '+')
for n in range(1, len(centres)+1):
    plt.annotate(n, (np.array(centres)[n-1:n,0], np.array(centres)[n-1:n,1]) , (np.array(centres)[n-1:n,0]+10, np.array(centres)[n-1:n,1]))

plt.savefig('numbered_centres.png')

In [ ]:
# now move the microscope to the position so that spectra can be taken
# this involves correcting the movement of the micrscope using template matching

# need to know the microscope's initial position
initial_position = microscope.position

# this function  finds the location of a previously defined template within an image
def find_template(template, image):
    corr = cv.matchTemplate(
        template,
        image,
        cv.TM_CCOEFF_NORMED
    )
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(corr)
    return [c + w//2 for c, w in zip(max_loc, template.shape[:2])]

    
fX, fY = 292, 190  # This is where the fibre appears
for (cX, cY) in centres:
    try:
        #make 100x100 template around centre point of the feature
        r = template_size // 2
        template = image[cY-r:cY+r, cX-r:cX+r]
    
        # check that the template location of the template in the image is where it should be
        max_loc = find_template(template, image)
        print(f"Centre is at {cX}, {cY}, max_loc is at {max_loc}")
        assert cX == max_loc[0] and cY == max_loc[1]
        
        new_image = microscope.grab_image_array()
        new_max_loc = find_template(template, new_image)
        dX = fX - new_max_loc[0]
        dY = fY - new_max_loc[1]
        print(f"New peak is at {new_max_loc}, so we should move {-dX}, {-dY}")
        plt.figure()
        plt.imshow(image)
        plt.plot([fX], [fY], "o")
        plt.plot([cX],[cY], "+")
        
    
        #move the microscope to the desired location
        print(f"Actually moving {cY-fY}, {cX-fX}")
        microscope.move_in_pixels(cY-fY, cX-fX)

        # check that the move has been accurate and correct it if not
        for n in range(5):
            microscope.autofocus()
            time.sleep(0.5)
            new_image = microscope.grab_image_array()
            new_max_loc = find_template(template, new_image)
            plt.figure()
            plt.imshow(new_image)
            plt.plot([fX], [fY], "o")
            plt.plot([new_max_loc[0]],[new_max_loc[1]], "+")
            dX = fX - new_max_loc[0]
            dY = fY - new_max_loc[1]
            if np.abs(dX) < 10 and np.abs(dY) < 10:
                print("Found it!")
                break
            else:
                print(f"New peak is at {new_max_loc}, so we should move {-dX}, {-dY}")
                microscope.move_in_pixels(-dY, -dX)
                continue

        position = centres.index((cX,cY)) +1
        #take and process spectrum
        take_spectrum(spec, 'spectrum{0}_{1}_{2}'.format(position,cX,cY), 'spectrumplot{0}_{1}_{2}.jpg'.format(position,cX,cY))
        process_spec('spectrum{0}_{1}_{2}'.format(position,cX,cY))
    
        #return the microscopes position to its original position
    finally:
        microscope.move([initial_position[k] for k in ['x', 'y', 'z']], absolute=True)
    